In [3]:
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm 
import json
import os
import datetime
import re
import csv
import time
import pandas as pd

In [ ]:
#TODO
#Download all the missing pages
#Create all the missing tsv
#Clean and comment the code
#CHECK EFFICIENCY 1.2 and 1.3
#Parse list before saving into

### 1. Data collection
## 1.1. Get the list of places

In [ ]:
total_link = []
s = requests.Session()
for n in tqdm(range(1,401)):
    url = f'https://www.atlasobscura.com/places?page={n}&sort=likes_count'
    result = s.get(url)
    soup = bs(result.text)
    puf = soup.find_all("a", {'class': 'content-card content-card-place'})
    for x in puf:
        total_link.append(x['href'])


In [ ]:
f = open('total_link.txt',"r")
total_link = f.read().split("\n")

## 1.2. Crawl places

In [ ]:
def downloadPage(start,end, array):
    count_link = ((start-1)*18)+1
    count_page = start
    
    parent_dir = f'./all_Pages'
    s = requests.Session()
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    for x in tqdm(range(start, end)):
        if count_link%10 == 0:
            s = requests.Session()
        if count_link%18 == 1:
            #directory = f'folder_{count_page}'
            path = os.path.join(parent_dir, f"folder_{count_page}")
            #os.remove(path)
            os.mkdir(path)
        for y in range(18):
            url = f'https://www.atlasobscura.com{array[count_link-1]}'
            name_file = f'location_{count_link}'
            name_folder = f'folder_{count_page}'
            with open(f'./all_Pages/{name_folder}/{name_file}.html', 'w', encoding='utf8') as fp:
                a = s.get(url, headers = header)
                fp.write(a.text)
                if a.status_code != 200:
                     time.sleep(120)
                     a = s.get(url, headers = header)
                fp.write(s.get(url).text)
            if count_link%18 == 0:
                count_page += 1
            count_link += 1
           
        

In [ ]:
arr = [47, 98, 89, 223, 56, 73,81, 307, 299, 182, 282, 198, 324, 215, 265, 173, 240, 190, 257, 383, 207, 316, 291, 249, 333, 358, 400, 374, 232, 349, 391, 274, 341]
arr = sorted(arr)
print(arr)

# for x in arr:
#     downloadPage(x, x+1, total_link)

## 1.3 Parse downloaded pages

In [ ]:


os.mkdir("TSV Files")
for x in os.listdir("all_Pages"):
    for y in os.listdir(f'all_Pages/{x}'):
        with open(f'./all_Pages/{x}/{y}', encoding='utf8') as f:
            p = f.read()
            soup =  bs(p)
            print(f"{x}/{y}")
            pageAttribute = []
            placeName = findPlaceName(soup)
            placeTags = findPlaceTags(soup)
            numPeopleVisited = findNumPeopleVisited(soup)
            numPeopleWant = findNumPeopleWant(soup)
            placeDesc = findDescription(soup)
            placeShortDesc = findShortDescription(soup)
            placeNearby = findNearbyPlaces(soup)
            placeAddress = findAddress(soup)
            placeAlt, placeLong = findCordinates(soup)
            placeEditors = findPostEditors(soup)
            placePubDate = findPublishingDate(soup)
            placeRelatedList = findPlaceNear(soup)
            placeRelatedPlaces = findRelatedPlaces(soup)
            placeURL = findPageURL(soup)
            with open(f'./TSV Files/{y[:-5]}.tvs', 'wt', encoding='utf8') as fp:
                tsv_writer = csv.writer(fp, delimiter='\t')
                tsv_writer.writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedList, placeRelatedPlaces, placeURL])
        

In [ ]:
def findPlaceName(soup):
    placeName = soup.find("h1", {"class": "DDPage__header-title"})
    if placeName != None:
        placeName = placeName.text
    #placeName = re.sub('[A-Za-z0-9_.,! "]*' ,'',placeName)
    return placeName

def findPlaceTags(soup):
    tags = []
    placeTags = soup.find_all("a", {"class": "itemTags__link js-item-tags-link"})
    for tag in placeTags:
        t = tag.text.replace("\n", "")
        #t = re.sub('[A-Za-z0-9 _.,!"]*','',t)
        tags.append(t)
    return tags

def findNumPeopleVisited(soup):
    peopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})
    if len(peopleVisited) > 0:
        peopleVisited = int(peopleVisited[0].text)
    return peopleVisited

def findNumPeopleWant(soup):
    peopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})
    if len(peopleVisited) > 0:
        peopleVisited = int(peopleVisited[1].text)
    return peopleVisited

def findDescription(soup):
    all_description = []
    descriptions = soup.find_all("div", {"class": "DDP__body-copy"})
    for description in descriptions:
        d = description.text.replace("\n","")
        all_description.append(d)
    return all_description

def findShortDescription(soup):
    shortDescription = soup.find("h3", {"class": "DDPage__header-dek"})
    if shortDescription != None:
        shortDescription = shortDescription.text.replace("\n", "")
    return shortDescription

def findNearbyPlaces(soup):
    nearPlaces = []
    nearbyPlaces = soup.find_all("div", {"class": "DDPageSiderailRecirc__item-title"})
    if nearbyPlaces != None:
        for place in nearbyPlaces:
            p = place.text.replace("\n","")
            nearPlaces.append(p)
        #Convert the list to set, and then back again to list to remove repetition
    return list(set(nearPlaces))

def findAddress(soup):
    strings = []
    adress_strings = soup.find("address", {"class": "DDPageSiderail__address"})
    if adress_strings != None:
        adress_strings = adress_strings.find("div")
        for info in adress_strings:
            s = info.text.replace("\n", "")
            if s != "":
                strings.append(s)
        if len(strings) > 3:
            return " ".join(strings[:3])
    else:
        return " "

def findCordinates(soup):

    cordinates = soup.find("div", {"class":"DDPageSiderail__coordinates js-copy-coordinates"})
    if cordinates != None:
        return cordinates.text.replace("\n", "").replace(" ","").split(",")
    return " ", " "

def findPostEditors(soup):
    all_editors =[]
    editors = soup.find_all("a", {"class":"DDPContributorsList__contributor"})         
    for person in editors:
        s = person.text.replace("\n", "")
        all_editors.append(s)
    return all_editors

def findPublishingDate(soup):
    #Pick the right info
    dateString = soup.find("div", {"class":"DDPContributor__name"}) 
    #Let's clean the string
    s = dateString.text.replace("\n", "")
    #Let's modify it for the right format of datetime
    split = s.split()
    #Let's convert the string Month into the corrispondent number by using "strptime()" 
    split[0] = str(datetime.datetime.strptime(split[0], '%B').month)
    #My format
    format = "%m %d, %Y"
    #Convert from String to datetime
    date = datetime.datetime.strptime(" ".join(split), format)
    return date

def findPlaceNear(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Nearby"})
    if relatedLists != None:
        relatedLists = relatedLists.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
        for list in relatedLists:
            s = list.text.replace("\n", "")
            #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
            lists.append(s)
    return lists

def findRelatedPlaces(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Related"})
    if relatedLists != None:
        relatedLists = relatedLists.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
        for list in relatedLists:
            s = list.text.replace("\n", "")
            #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
            lists.append(s)
    return lists

def findPageURL(soup):
    numVisitedPeople = soup.find("link", {"rel":"canonical"})
    return numVisitedPeople['href']



## Load all the tsv file into one pandas dataframe

In [1]:

def load_tsv():
    tsv = []
    dtypes = {}
    for x in os.listdir("TSV Files"):
        df = pd.read_csv(f'TSV Files/{x}',
            sep="\t",
            header=None,
            names=["placeName", "placeTags", "numPeopleVisited", "numPeopleWant", "placeDesc", "placeShortDesc", "placeNearby","placeAdress", "placeAlt", "placeLong", "placeEditors","placePubDate", "placeRelatedList", "placeRelatedPlace", "placeURL"])
        tsv.append(df)

    return pd.concat(tsv)

In [4]:
data = load_tsv()


In [5]:
#Reset Index
data.reset_index(inplace = True)
data = data.drop(["index"], axis = 1)
data.head()


,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedList,placeRelatedPlace,placeURL
0,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1834,8603,['The first New York City subway was built and...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"['Rebekah Otto', 'Annetta Black', 'Allan', 'ch...",2010-05-08 00:00:00,"['African Burial Ground National Monument', 'T...","['Crystal Palace Subway', 'Moscow Metro Statio...",https://www.atlasobscura.com/places/city-hall-...
1,Dream House,"['optical oddities', 'eccentric homes', 'music...",765,6007,"['When walking down Church Street in Tribeca, ...","La Monte Young and Marian Zazeela's ""Dream Hou...","['Aretha Franklin Subway Tributes', ""The 'Ghos...","275 Church St New York, New York, 10013 United...",40.7185,-74.0048,"['seanmattison', 'mbison', 'Gray', 'Sofy', 'er...",2009-09-22 00:00:00,"['Aretha Franklin Subway Tributes', 'Farm.One'...","['Callejon de Hamel', 'Casa Neverlandia', 'Cas...",https://www.atlasobscura.com/places/dream-house
2,The Museum of Death,"['crime and punishment', 'memento mori', 'deat...",1761,5832,"['The Museum of Death, a showcase of just that...",World's largest collection of serial killer ar...,"['Capitol Records Building Morse Code', 'Gower...","6031 Hollywood Blvd. Los Angeles, California, ...",34.1017,-118.3212,"['Genki', 'hana', 'Lauren Levesque', 'britbarb...",2010-08-09 00:00:00,"['Hollywood Tower Apartments', 'Capitol Record...","['Museum Tot Zover', ' Roman Urn Collection at...",https://www.atlasobscura.com/places/the-museum...
3,God's Own Junkyard,"['neon', 'signs', 'collections', 'art']",856,5812,"['For years, Walthamstow was known as an unrem...",A kaleidoscopic warehouse-maze of handmade neo...,"['The Ancient House', 'Walthamstow Market', 'T...","82A Shernhall St London, England United Kingdom",51.5839,-0.0083,"['FrustratedFormerContributor', 'ryanswift', '...",2017-04-18 00:00:00,"['The Ancient House', 'The Bremer Car', 'Walth...","['Ignite Sign Art Museum', 'Buchstabenmuseum '...",https://www.atlasobscura.com/places/gods-own-j...
4,The Last Bookstore,"['bookstores', 'repositories of knowledge', 'b...",3614,5816,['More than a few bibliophiles would prefer th...,This iconic L.A. bookshop is housed in an aban...,"['Biddy Mason Memorial Park', 'Bradbury Buildi...","453 S. Spring St. Los Angeles, California, 900...",34.0477,-118.2494,"['kcmick', 'paulruta', 'Avoiding Regret', 'Jes...",2013-02-27 00:00:00,"['Biddy Mason Memorial Park', 'Bradbury Buildi...","['Alabama Booksmith', 'Book Corner', 'John K. ...",https://www.atlasobscura.com/places/last-books...


In [6]:
#The first dictionary where every key is a word and it's value is the corrispondant number to that word

def createFirstDic(data):
    dic = {}
    value = 1
    for description in data["placeDesc"]:
        for word in description.split():
            if word in dic.keys():
                continue
            else:
                dic[word] = value
                value += 1
    return dic

In [7]:
dic = createFirstDic(data)

In [8]:
dic

{"['The": 1,
 'first': 2,
 'New': 3,
 'York': 4,
 'City': 5,
 'subway': 6,
 'was': 7,
 'built': 8,
 'and': 9,
 'operated': 10,
 'by': 11,
 'the': 12,
 'Interborough': 13,
 'Rapid': 14,
 'Transit': 15,
 'Company': 16,
 '(IRT)': 17,
 'opened': 18,
 'on': 19,
 'October': 20,
 '27,': 21,
 '1904,': 22,
 'to': 23,
 'joy': 24,
 'of': 25,
 'elevated': 26,
 'train': 27,
 'streetcar': 28,
 'riders.The': 29,
 'Hall': 30,
 'station': 31,
 'IRT': 32,
 'local': 33,
 'track': 34,
 'lavished': 35,
 'with': 36,
 'fine': 37,
 'architectural': 38,
 'details,': 39,
 'including': 40,
 'glass': 41,
 'tiles': 42,
 'large': 43,
 'chandeliers.': 44,
 'However,': 45,
 'Gustavino': 46,
 'vaulted': 47,
 'ceilings': 48,
 'skylights': 49,
 'were': 50,
 'lost': 51,
 'busy': 52,
 'commuters,': 53,
 'stop': 54,
 'one': 55,
 'least-used': 56,
 'in': 57,
 'system.': 58,
 'It': 59,
 'only': 60,
 'that': 61,
 'did': 62,
 'not': 63,
 'have': 64,
 'turnstiles': 65,
 'installed': 66,
 '1923,': 67,
 'nearby': 68,
 'Brooklyn':